# Chapter8. 조인

## 8.1 조인 표현식

스파크의 조인 방식 : 왼쪽과 오른쪽 데이터셋에 있는 하나 이상의 키 값을 비교해 왼쪽 데이터셋과 오른쪽 데이터셋의 결합 여부를 결정하는 조인 표현식의 평가 결과에 따름.

* 동등 조인(equi-join) : 왼쪽과 오른쪽 데이터셋에 지정된 키가 동일한지 비교 - 키가 일치하면 왼쪽과 오른쪽 데이터셋 결합(일치하지 않으면 결합 X)

## 8.2 조인 타입

< 스파크에서 사용할 수 있는 조인 타입 >
* 내부 조인(inner join) : 왼쪽과 오른쪽 데이터셋에 키가 있는 로우 유지
* 외부 조인(outer join) : 왼쪽이나 오른쪽 데이터셋에 키가 있는 로우 유지
* 왼쪽 외부 조인(left outer join) : 왼쪽 데이터셋에 키가 있는 로우 유지
* 오른쪽 외부 조인(right outer join) : 오른쪽 데이터셋에 키가 있는 로우 유지
* 왼쪽 세미 조인(left semi join) : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우 키가 일치하는 왼쪽 데이터셋만 유지
* 왼쪽 안티 조인(left anti join) : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우 키가 일치하지 않는 왼쪽 데이터셋만 유지
* 자연 조인(natural join) : 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인
* 교차 조인(cross join) 또는 카테시안 조인(catesian join) : 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터셋의 모든 로우 조합

코드) 예제에서 사용할 간단한 데이터셋 생성

In [0]:
person = spark.createDataFrame([
  (0, "Bill Chambers", 0, [100]),
  (1, "Matei Zaharia", 1, [500, 250, 100]),
  (2, "Michael Armbrust", 1, [250, 100])
]).toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
  (0, "Masters", "School of Information", "UC Berkeley"),
  (2, "Masters", "EECS", "UC Berkeley"),
  (1, "Ph.D", "EECS", "UC Berkeley")
]).toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
  (500, "Vice President"),
  (250, "PMC Member"),
  (100, "Contributor")
]).toDF("id", "status")

In [0]:
# 위에서 생성한 데이터셋을 테이블로 등록

person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

## 8.3 내부 조인

내부 조인은 DataFrame이나 테이블에 존재하는 키를 평가해 true로 평가되는 로우만 결합

예제) graduateProgram DataFrame과 person DataFrame을 조인해 새로운 DataFrame 생성

In [0]:
joinExpression = person["graduate_program"] == graduateProgram['id']

In [0]:
# 두 DataFrame 모두에 키가 존재하지 않으면 결과 DataFrame에서 볼 수 없음.

wrongJoinExpression = person["name"] == graduateProgram["school"]

내부 조인은 기본 조인 방식이므로 JOIN 표현식에 왼쪽 DataFrame과 오른쪽 DataFrame을 지정.

In [0]:
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

join 메서드의 세 번째 파라미터(joinType)로 조인 타입 지정 가능.

In [0]:
joinType = "inner"

person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

## 8.4 외부 조인

외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가해 true나 false로 평가한 로우를 포함해 조인. 

단, 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null 삽입

In [0]:
joinType = "outer"

person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

## 8.5 왼쪽 외부 조인

왼족 외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가해 왼쪽 DataFrame의 모든 로우와 왼쪽 DataFrame과 일치하는 오른쪽 DataFrame의 로우를 함께 포함.

단, 오른쪽 DataFrame에 일치하는 로우가 없으면 해당 위치에 null 삽입.

In [0]:
joinType = "left_outer"

graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 2|Masters| EECS|UC Berkeley|null| null| null| null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+

## 8.6 오른쪽 외부 조인

오른쪽 외부 조인은 오른쪽 DataFrame의 모든 로우와 오른쪽 DataFrame과 일치하는 왼쪽 DataFrame의 로우를 함께 포함.

단, 왼쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null 삽입.

In [0]:
joinType = "right_outer"

person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

## 8.7 왼쪽 세미 조인

세미 조인은 오른쪽 DataFrame의 어떤 값도 포함하지 않기 때문에 다른 조인 타입과 다르지만 두 번째 DataFrame은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용. 

만약 값이 존재한다면 왼쪽 DataFrame에 중복 키가 존재하더라도 해당 로우는 결과에 포함.

즉, 기존 조인 기능과 달리 DataFrame의 필터 의미.

In [0]:
joinType = "left_semi"

graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
 id| degree| department| school|
+---+-------+--------------------+-----------+
 0|Masters|School of Informa...|UC Berkeley|
 1| Ph.D| EECS|UC Berkeley|
+---+-------+--------------------+-----------+

In [0]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
  (0, "Masters", "Duplicated Row", "Duplicated School")
]))

gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------------+
 id| degree| department| school|
+---+-------+--------------------+-----------------+
 0|Masters|School of Informa...| UC Berkeley|
 1| Ph.D| EECS| UC Berkeley|
 0|Masters| Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+

## 8.8 왼쪽 안티 조인

왼쪽 안티 조인은 왼쪽 세미 조인의 반대 개념.

왼쪽 세미 조인처럼 오른쪽 DataFrame의 어떤 값도 포함하지만 두 번째 DataFrame은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용.

하지만 두 번째 DataFrame에 존재하는 값을 유지하는 대신 두 번째 DataFrame에서 관련된 키를 찾을 수 없는 로우만 결과에 포함.

(안티 조인은 SQL의 NOT IN과 같은 의미)

In [0]:
joinType = "left_anti"

graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
 id| degree|department| school|
+---+-------+----------+-----------+
 2|Masters| EECS|UC Berkeley|
+---+-------+----------+-----------+

## 8.9 자연 조인

자연 조인은 조인하려는 컬럼을 암시적으로 추정. 

즉, 일치하는 컬럼을 찾고 그 결과를 반환.

## 8.10 교차 조인(카테시안 조인)

교차 조인은 조건절을 기술하지 않은 내부 조인 의미. 

왼쪽 DataFrame의 모든 로우를 오른쪽 DataFrame의 모든 로우와 결합하기 때문에 교차 조인을 거치면 엄청난 수의 로우를 가진 DataFrame이 생성될 수 있음.

ex) 1,000개의 로우가 존재하는 두 개의 DataFrame에 교차 조인을 수행하면 1,000 X 1,000 = 1,000,000개의 결과 로우 생성.

따라서 반드시 키워드를 이용해 교차 조인 수행해야함.

In [0]:
joinType = "cross"

graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

In [0]:
# 교차 조인 메서드 호출 방식 사용 가능

person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 0| Bill Chambers| 0| [100]| 2|Masters| EECS|UC Berkeley|
 0| Bill Chambers| 0| [100]| 1| Ph.D| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 2|Masters| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 2|Masters| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

## 8.11 조인 사용시 문제점

### 8.11.1 복합 데이터 타입의 조인

불리언을 반환하는 모든 표현식은 조인 표현식으로 간주 가능.

In [0]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personID").join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
personID| name|graduate_program| spark_status| id| status|
+--------+----------------+----------------+---------------+---+--------------+
 0| Bill Chambers| 0| [100]|100| Contributor|
 1| Matei Zaharia| 1|[500, 250, 100]|500|Vice President|
 1| Matei Zaharia| 1|[500, 250, 100]|250| PMC Member|
 1| Matei Zaharia| 1|[500, 250, 100]|100| Contributor|
 2|Michael Armbrust| 1| [250, 100]|250| PMC Member|
 2|Michael Armbrust| 1| [250, 100]|100| Contributor|
+--------+----------------+----------------+---------------+---+--------------+

### 8.11.2 중복 컬럼명 처리

DataFrame의 각 컬럼은 스파크 SQL 엔진인 카탈리스트 내에 고유 ID 존재.

고유 ID는 카탈리스트 내부에서만 사용이 가능하며 직접 참조 가능한 값은 아님. 

따라서 중복된 컬럼명이 존재하는 DataFrame을 사용할 때는 특정 컬럼을 참조하기 매우 어렵다.

* 상황1. 조인에 사용할 DataFrame의 특정 키가 동일한 이름을 가지며, 키가 제거되지 않도록 조인 표현식에 명시하는 경우
* 상황2. 조인 대상이 아닌 두 개의 컬럼이 동일한 이름을 가진 경우

코드) 잘못된 데이터셋 생성

In [0]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]

In [0]:
person.join(gradProgramDupe, joinExpr).show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|graduate_program| degree| department| school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+

위의 코드 실행 결과 graduate_program 컬럼을 키로 조인했음에도 불구하고 두 개의 graduate_program 컬럼 존재

이러한 컬럼 중 하나를 참조할 때 문제 발생.

In [0]:
person.join(gradProgramDupe, joinExpr).select("graduate_program").show()  # 오류 발생

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-31866354770816> in <module> 
 ----> 1 person . join ( gradProgramDupe , joinExpr ) . select ( "graduate_program" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.

**해결방법 1 : 다른 조인 표현식 사용**

불리언 형태의 조인 표현식을 문자열이나 시퀀스 형태로 변경 => 조인을 할 때 두 컬럼 중 하나가 자동 제거됨.

In [0]:
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

**해결방법 2 : 조인 후 컬럼 제거**

조인 후에 문제가 되는 컬럼을 제거

단, 원본 DataFrame을 사용해 컬럼을 참조해야 함. 

=> 조인 시 동일한 키 이름을 사용하거나 원본 DataFrame에 동일한 컬럼이 존재하는 경우에 사용 가능.

In [0]:
person.join(gradProgramDupe, joinExpr).drop(person["graduate_program"]).select("graduate_program").show()

+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

**해결방법 3 : 조인 전 컬럼명 변경**

조인 전에 컬럼명을 변경하면 문제 회피 가능.

In [0]:
gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
joinExpr = person["graduate_program"] == gradProgram3["grad_id"]
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|grad_id| degree| department| school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D| EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+

## 8.12 스파크의 조인 수행 방식

< 두 가지 핵심 전략 >
* 노드간 네트워크 통신 전략
* 노드별 연산 전략

### 8.12.1 네트워크 통신 전략

< 두 가지 클러스터 통신 방식 활용 >

* 셔플 조인(shuffle join) - 전체 노드 간 통신 유발 O
* 브로드캐스트 조인(broadcast join) - 전체 노드 간 통신 유발 X

**큰 테이블과 큰 테이블 조인**

셔플 조인은 전체 노드 간 통신이 발생하고 조인에 사용한 특정 키나 키 집합을 어떤 노드가 가졌는지에 따라 해당 노드와 데이터 공유. 

이런 방식 때문에 네트워크는 복잡해지고 많은 자원 사용. (특히 데이터가 잘 나뉘어 있지 않다면 더 심해짐.)

셔플 조인 과정은 큰 테이블의 데이터를 다른 큰 테이블의 데이터와 조인하는 과정을 잘 나타탬.

ex. 사물인터넷 환경에서 매일 수십억 개의 메시지를 수신하고 일별 변경사항을 식별해야 한다면 deviceId, messageType 그리고 data와 data-1을 나타내는 컬럼을 이용해 조인. 

**큰 테이블과 작은 테이블 조인**

테이블이 단일 워커노드의 메모리 크기에 적합할 정도(메모리 여유 공간 포함)로 충분히 작은 경우 조인 연산 최적화 가능.

큰 테이블 사이의 조인에 사용한 방법도 유용하지만 브로드캐스트 조인이 훨씬 효율적.

브로드캐스트 조인은 작은 DataFrame을 클러스터의 전체 워커 노드에 복제하는 것 의미. 

브로드캐스트 조인은 이전 조인 방식과 마찬가지로 대규모 노드 간 통신이 발생하지만 그 이후로는 노드 사이에 추가적인 통신 발생 X.

따라서 모든 단일 노드에서 개별적으로 조인이 수행되므로 CPU가 가장 큰 병목 구간으로 된다.

**아주 작은 테이블 사이의 조인**

아주 작은 테이블 사이의 조인을 할 때는 스파크가 조인 방식을 결정하도록 내버려두는 것이 제일 좋다. 

하지만 필요한 경우 브로드캐스트 조인을 강제 지정 가능.